El set de datos de entrenamiento fue sometido previamente a un ajuste de dimensiones para pasar de los originales 181x217x181 a 192x192x192.

#**INSTALACIONES NECESARIAS**

In [ ]:
!pip install segmentation-models # Modelos de segmentación preentrenados para tareas de visión por computadora
!pip install nibabel # Carga y manipulación de imágenes médicas en formato NIfTI
!pip install volumentations-3D # Aumentaciones de datos específicas para imágenes 3D

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.1 MB/s eta 0:00:00


##**IMPORTAMOS LIBRERIAS NECESARIAS**

In [ ]:
# Importar librerías

import os
import nibabel as nib
import numpy as np
import random
import glob
import tensorflow as tf
from tensorflow import keras
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
# CONFIGURAMOS KERAS COMO BACKEND DE segmentation_models
sm.set_framework('tf.keras')
sm.framework()
import matplotlib.pyplot as plt
# Modelos, capas, backend, callbacks, regularizadores y pérdidas para la red neuronal.
from tensorflow.keras import models, layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (Input, Activation, BatchNormalization, SpatialDropout3D,
                                     Conv3D, MaxPooling3D, UpSampling3D, Conv3DTranspose, concatenate)
from tensorflow.data.experimental import AUTOTUNE
from tensorflow.keras.losses import BinaryFocalCrossentropy
from segmentation_models.metrics import IOUScore, FScore, Precision, Recall
from tensorflow.keras.losses import Dice
from sklearn.model_selection import KFold
from volumentations import Compose, Flip, RandomRotate90


Segmentation Models: using `tf.keras` framework.


#**DEFINIMOS LA FUNCIÓN DE PERDIDA**

In [ ]:
# DEFINIMOS DICE LOSS
dice_loss = Dice(reduction='sum_over_batch_size',name='dice') # El argumento reduction='sum_over_batch_size' especifica cómo se agrega la pérdida en todo el lote.
# La función de pérdida Dice mide la superposición entre las máscaras de segmentación predichas y reales.

In [ ]:
# DEFINIMOS PERDIDA FOCAL BINARIA PARA MANEJAR EL DESBALANCE DE CLASES (LESIONES Y FONDO)
focal_loss = BinaryFocalCrossentropy(gamma=2.0)
# Configura la pérdida de Entropía Cruzada Focal Binaria, que aborda el desequilibrio de clases asignando más peso a los píxeles difíciles de clasificar.

In [ ]:
# FUNCIÓN DE PERDIDA COMBINADA
def combined_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + focal_loss(y_true, y_pred)
# La función combina la pérdida Dice y la pérdida Focal para crear una función de pérdida más robusta. Al sumar ambas pérdidas, se alienta al modelo a optimizar tanto la precisión
# de la superposición (pérdida Dice) como el desequilibrio de clases (pérdida Focal).

#**DEFINIMOS METRICAS DE OPTIMIZACIÓN**

In [ ]:
# COEFICIENTE DE DICE
dice_metric = FScore(beta=1, threshold=0.5, name='dice_metric')
# DSC, también conocido como F1-score. El argumento threshold=0.5 especifica que las predicciones superiores a 0.5 se consideran positivas que tan bien coincide la máscara
# predicha con el ground truth

In [ ]:
# INDICE DE JACCARD
jaccard_metric = IOUScore(threshold=0.5, name='jaccard_metric') # Define el índice de Jaccard (Intersección sobre Unión, IoU) como métrica, también con un umbral de 0.5.
# PRECISION, SENCIBILIDAD Y ESPECIFICIDAD
precision_metric = Precision(threshold=0.5, name='precision_metric') # Indica la proporción de casos positivos predichos correctamente del total de casos positivos predichos.
recall_metric = Recall(threshold=0.5, name='recall_metric') # Midie la proporción de casos positivos predichos correctamente del total de casos positivos reales.

#**DEFINIMOS LA ARQUITECTURA U-Net 3D**



In [ ]:
# Parámetros del modelo
bnorm_axis = -1 # Especifica el eje para la normalización por lotes. Un valor de -1 indica que la normalización por lotes debe aplicarse sobre el último eje (canales).
nfilters = np.array([32, 48, 96, 192, 384]) # Experimentación
drop_rate = 0.2 # Tasa de dropout para las capas SpatialDropout3D (para evitar el sobreajuste)
input_shape = (192, 192, 192, 1)

def unet_3d_version10(input_shape):
    inputs = Input(shape=input_shape)

    # ============== Encoder ==============
    # Encoder Block 0
    e0 = Conv3D(filters=nfilters[0], kernel_size=(3,3,3), padding='same')(inputs)
    e0 = BatchNormalization(axis=bnorm_axis)(e0)
    e0 = Activation('relu')(e0)
    e0 = Conv3D(filters=nfilters[0], kernel_size=(3,3,3), padding='same')(e0)
    e0 = BatchNormalization(axis=bnorm_axis)(e0)
    e0 = Activation('relu')(e0)

    # Encoder Block 1
    e1 = MaxPooling3D((2,2,2))(e0)
    e1 = Conv3D(nfilters[1], (3,3,3), padding='same')(e1)
    e1 = BatchNormalization(axis=bnorm_axis)(e1)
    e1 = Activation('relu')(e1)
    e1 = Conv3D(nfilters[1], (3,3,3), padding='same')(e1)
    e1 = BatchNormalization(axis=bnorm_axis)(e1)
    e1 = Activation('relu')(e1)

    # Encoder Block 2
    e2 = SpatialDropout3D(drop_rate)(e1, training=True)
    e2 = MaxPooling3D((2,2,2))(e2)
    e2 = Conv3D(nfilters[2], (3,3,3), padding='same')(e2)
    e2 = BatchNormalization(axis=bnorm_axis)(e2)
    e2 = Activation('relu')(e2)
    e2 = Conv3D(nfilters[2], (3,3,3), padding='same')(e2)
    e2 = BatchNormalization(axis=bnorm_axis)(e2)
    e2 = Activation('relu')(e2)

    # Encoder Block 3
    e3 = SpatialDropout3D(drop_rate)(e2, training=True)
    e3 = MaxPooling3D((2,2,2))(e3)
    e3 = Conv3D(nfilters[3], (3,3,3), padding='same')(e3)
    e3 = BatchNormalization(axis=bnorm_axis)(e3)
    e3 = Activation('relu')(e3)
    e3 = Conv3D(nfilters[3], (3,3,3), padding='same')(e3)
    e3 = BatchNormalization(axis=bnorm_axis)(e3)
    e3 = Activation('relu')(e3)

    # ============== Bottleneck ==============
    e4 = SpatialDropout3D(drop_rate)(e3, training=True)
    e4 = MaxPooling3D((2,2,2))(e4)
    e4 = Conv3D(nfilters[4], (3,3,3), padding='same')(e4)
    e4 = BatchNormalization(axis=bnorm_axis)(e4)
    e4 = Activation('relu')(e4)
    e4 = Conv3D(nfilters[4], (3,3,3), padding='same')(e4)
    e4 = BatchNormalization(axis=bnorm_axis)(e4)
    e4 = Activation('relu')(e4)

    # ============== Decoder ==============
    # Decoder Block 3
    d3 = SpatialDropout3D(drop_rate)(e4, training=True)
    d3 = UpSampling3D((2,2,2))(d3)
    d3 = concatenate([e3, d3])
    d3 = Conv3DTranspose(nfilters[3], (3,3,3), padding='same')(d3)
    d3 = BatchNormalization(axis=bnorm_axis)(d3)
    d3 = Activation('relu')(d3)
    d3 = Conv3DTranspose(nfilters[3], (3,3,3), padding='same')(d3)
    d3 = BatchNormalization(axis=bnorm_axis)(d3)
    d3 = Activation('relu')(d3)

    # Decoder Block 2
    d2 = SpatialDropout3D(drop_rate)(d3, training=True)
    d2 = UpSampling3D((2,2,2))(d2)
    d2 = concatenate([e2, d2])
    d2 = Conv3DTranspose(nfilters[2], (3,3,3), padding='same')(d2)
    d2 = BatchNormalization(axis=bnorm_axis)(d2)
    d2 = Activation('relu')(d2)
    d2 = Conv3DTranspose(nfilters[2], (3,3,3), padding='same')(d2)
    d2 = BatchNormalization(axis=bnorm_axis)(d2)
    d2 = Activation('relu')(d2)

    # Decoder Block 1
    d1 = UpSampling3D((2,2,2))(d2)
    d1 = concatenate([e1, d1])
    d1 = Conv3DTranspose(nfilters[1], (3,3,3), padding='same')(d1)
    d1 = BatchNormalization(axis=bnorm_axis)(d1)
    d1 = Activation('relu')(d1)
    d1 = Conv3DTranspose(nfilters[1], (3,3,3), padding='same')(d1)
    d1 = BatchNormalization(axis=bnorm_axis)(d1)
    d1 = Activation('relu')(d1)

    # Decoder Block 0
    d0 = UpSampling3D((2,2,2))(d1)
    d0 = concatenate([e0, d0])
    d0 = Conv3DTranspose(nfilters[0], (3,3,3), padding='same')(d0)
    d0 = BatchNormalization(axis=bnorm_axis)(d0)
    d0 = Activation('relu')(d0)
    d0 = Conv3DTranspose(nfilters[0], (3,3,3), padding='same')(d0)
    d0 = BatchNormalization(axis=bnorm_axis)(d0)
    d0 = Activation('relu')(d0)

    # Capa de salida
    outputs = Conv3D(1, (1,1,1), activation='sigmoid', padding='same')(d0)

    # Compilación
    # Usamos combined_loss (dice_loss + focal_loss) y las métricas definidas en la sección anterior
    optimizer = tf.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizer, loss=combined_loss, metrics=[dice_metric, jaccard_metric, precision_metric, recall_metric])

    model.summary()
    return model



#**FUNCIONES AUXILIARES**

In [ ]:
#####################################
# Define un conjunto de aumentos aleatorios para aplicar a las imágenes y máscaras. Selecciona aleatoriamente volteos y rotaciones de 90 grados a lo largo de diferentes ejes.
# Esta función se utiliza en la función apply_augmentation, que se llama durante el pipeline de entrenamiento.
#####################################
def get_random_augmentation():
    augmentations = []
    applied_augmentations = []

    if random.random() < 0.5:
        augmentations.append(Flip(0, p=1.0))
        applied_augmentations.append('Flip en X')
    if random.random() < 0.5:
        augmentations.append(Flip(1, p=1.0))
        applied_augmentations.append('Flip en Y')
    if random.random() < 0.5:
        augmentations.append(Flip(2, p=1.0))
        applied_augmentations.append('Flip en Z')

    if random.random() < 0.5:
        augmentations.append(RandomRotate90((0, 1), p=1.0))
        applied_augmentations.append('RandomRotate90 en XY')
    if random.random() < 0.5:
        augmentations.append(RandomRotate90((1, 2), p=1.0))
        applied_augmentations.append('RandomRotate90 en YZ')
    if random.random() < 0.5:
        augmentations.append(RandomRotate90((0, 2), p=1.0))
        applied_augmentations.append('RandomRotate90 en XZ')

    if len(augmentations) == 0:
        random_choice = random.choice([
            (Flip(0, p=1.0), 'Flip en X'),
            (Flip(1, p=1.0), 'Flip en Y'),
            (Flip(2, p=1.0), 'Flip en Z'),
            (RandomRotate90((0, 1), p=1.0), 'RandomRotate90 en XY'),
            (RandomRotate90((1, 2), p=1.0), 'RandomRotate90 en YZ'),
            (RandomRotate90((0, 2), p=1.0), 'RandomRotate90 en XZ')
        ])
        augmentations.append(random_choice[0])
        applied_augmentations.append(random_choice[1])

    return Compose(augmentations, p=1.0), applied_augmentations

In [ ]:
#####################################
# Realiza dropout de Monte Carlo para estimar la incertidumbre de las predicciones del modelo. Ejecuta el modelo varias veces con dropout habilitado y
# devuelve la media y la varianza de las predicciones.
# Esta función se utiliza durante la fase de evaluación para analizar la incertidumbre de las predicciones del modelo.
#####################################
def predict_with_uncertainty(model, images, n_iter=1, batch_size=1):
    num_images = images.shape[0] # Numero total de imagenes (numero de muestras)
    all_preds = [] # Almacena las predicciones
    for _ in range(n_iter): # bucle a ejecutar n veces
        batch_preds = [] #Almacena las iteracciones para cada lote de imagenes
        for i in range(0, num_images, batch_size): # Segundo Bucle, comienza en 0 y aumentan en batch_size hasta llegar al numero de imagenes
            batch_images = images[i:i + batch_size] # Extrae un lote de imagenes usando los indices "i" del bucle anterior y lo guarda
            pred = model(batch_images, training=True) # Realiza prediccion usando el modelo en el lote (training=True activa dropout en entrenamiento) y lo guarda
            batch_preds.append(pred.numpy()) # Convierte pred a numpy y lo carga a la lista batch_preds
        all_preds.append(np.concatenate(batch_preds, axis=0)) #Cuando se procesan todos los lotes de la iteracion, concatena las predicciones y las guarda
    all_preds = np.array(all_preds) #Despues de todas las iteraciones all_preds se convierte a numpy (n_iter/#_imagenes/forma de salida del modelo[192,192,192,1])
    mean_preds = np.mean(all_preds, axis=0) # Calcula la media de las predicciones en el eje 0 (eje de las iteraciones)-> PREDICCION PROMEDIO
    var_preds = np.std(all_preds, axis=0) #np.var(all_preds, axis=0) # Calcula la varianza de las predicciones en el eje 0 (eje de las iteraciones)-> INCERTIDUMBRE PREDICCIONES
    return mean_preds, var_preds, all_preds # RETORNA LAS PREDICCIONES PROMEDIO, INCERTIDUMBRE DE LAS PREDICCIONES Y TODAS LAS PREDICCIONES EN CADA ITERACION.

In [ ]:
## OTRAS FUNCIONES AUXILIARES

#####################################
# Carga una imagen NIfTI desde la ruta de archivo especificada usando nib.load y devuelve los datos de la imagen y la matriz de transformación afín.
# Esta función se utiliza en el pipeline.
#####################################
def load_nifti_image(filepath):
    img = nib.load(filepath)
    return img.get_fdata(), img.affine

#####################################
# Normaliza la imagen restando la media y dividiendo por la desviación estándar. También binariza la máscara estableciendo los valores mayores o iguales a 0.5 a 1
# y los valores menores a 0.5 a 0.
# Esta función se utiliza en el pipeline.
#####################################
def preprocess_image(image, mask):
    # Normalizar la imagen (media cero y std uno)
    image = (image - np.mean(image)) / (np.std(image) + 1e-7)
    # Binarizar la máscara
    mask = np.where(mask >= 0.5, 1, 0)
    return image, mask

#####################################
# Aplica los aumentos seleccionados aleatoriamente a la imagen y máscara de entrada usando la librería volumentations.
# Esta función se utiliza durante el pipeline de entrenamiento para aumentar la variabilidad de los datos de entrenamiento.
#####################################
def apply_augmentation(img, msk):
    aug, applied_augmentations = get_random_augmentation()
    augmented = aug(image=img, mask=msk)
    return augmented['image'], augmented['mask']

#####################################
# Carga pares de imágenes y máscaras desde la ruta de carpeta especificada. Busca imágenes preprocesadas y máscaras correspondientes en los subdirectorios de la carpeta principal.
# Esta función se utiliza para cargar los datos para entrenamiento, validación y prueba.
#####################################

def load_images_masks(folder_path):
    """
    Devuelve lista con pares (imagen, mascara) Filtrar subdirectorios y ceros en create_dataset con la parte 'filtered_pairs'.
    """
    image_mask_pairs = []
    # Recorremos la carpeta principal
    for subject in os.listdir(folder_path):
        subject_folder      = os.path.join(folder_path, subject)
        preprocessed_folder = os.path.join(subject_folder, 'preprocessed')
        masks_folder        = os.path.join(subject_folder, 'masks')
        # Revisar que existan
        if not (os.path.isdir(preprocessed_folder) and os.path.isdir(masks_folder)):
            print("[WARNING] -> Subcarpeta inválida:", subject_folder)
            continue
        for file in os.listdir(preprocessed_folder):
            if '_flair_pp_padded.nii' in file:
                flair_image_path = os.path.join(preprocessed_folder, file)
                mask_name        = file.replace('_flair_pp_padded.nii','_mask1_padded.nii')
                mask_path        = os.path.join(masks_folder, mask_name)
                if os.path.exists(mask_path) and os.path.isfile(flair_image_path):
                    image_mask_pairs.append((flair_image_path, mask_path))
                else:
                    print("[WARNING] -> No existe la máscara o la imagen:", flair_image_path, mask_path)

    return image_mask_pairs


###########################################
# Esta es una función interna que realiza la carga y el aumento de datos reales. Toma las rutas de archivo de la imagen y la máscara como entrada, carga los datos usando
# load_nifti_image, preprocesa los datos usando preprocess_image y aplica aumentos si el indicador augment está configurado como True.
# Esta función se utiliza dentro de tf_load_preprocess.
###########################################

def _py_load_augment(f_str_tensor, m_str_tensor, augment):
    """Función interna que hace la carga (nib.load) + volumentations."""
    # Convertir EagerTensor -> str real
    f_str = f_str_tensor.numpy().decode('utf-8')
    m_str = m_str_tensor.numpy().decode('utf-8')
    # 1) Carga nibabel
    img_data, _ = load_nifti_image(f_str)
    msk_data, _ = load_nifti_image(m_str)
    # 2) Preprocesar
    img_data, msk_data = preprocess_image(img_data, msk_data)
    # 3) Augment si es True
    if augment:
        img_data, msk_data = apply_augmentation(img_data, msk_data)
    # 4) Expandir dims
    img_data = np.expand_dims(img_data, axis=-1)
    msk_data = np.expand_dims(msk_data, axis=-1)

    return img_data, msk_data

###########################################
# Esta función envuelve la función _py_load_augment para que sea compatible con la API tf.data de TensorFlow. Utiliza tf.py_function para ejecutar la función de Python
# dentro del grafo de TensorFlow.
# Esta función se utiliza para cargar y preprocesar los datos en paralelo.
###########################################
def tf_load_preprocess(f_p, m_p, augment):
    """Función principal que se llama desde create_dataset."""
    [img, msk] = tf.py_function(
        func=_py_load_augment,
        inp=[f_p, m_p, augment],
        Tout=[tf.float32, tf.float32]
    )
    # Ajustamos shape fija
    img.set_shape([192, 192, 192, 1])
    msk.set_shape([192, 192, 192, 1])
    return img, msk


###########################################
# Crea un conjunto de datos de TensorFlow a partir de una lista de pares de imagen-máscara. Mezcla los datos, aplica la función tf_load_preprocess para cargar y
# preprocesar los datos, divide los datos en lotes y realiza una búsqueda previa de los datos para mejorar el rendimiento.
# Esta función se utiliza para crear los conjuntos de datos de entrenamiento, validación y prueba.
###########################################

def create_dataset(image_mask_pairs, augment=False, batch_size=1):
    """Reemplazo de tu create_dataset usando lo anterior."""
    flair_paths = [pair[0] for pair in image_mask_pairs]
    mask_paths  = [pair[1] for pair in image_mask_pairs]

    ds = tf.data.Dataset.from_tensor_slices((flair_paths, mask_paths))
    ds = ds.shuffle(buffer_size=len(flair_paths))

    # Mapeamos con la closure que inyecta 'augment'
    ds = ds.map(lambda f,m: tf_load_preprocess(f, m, augment),
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

#####################################
# FUNCIONES DE DEPURACIÓN
# Verifica si la imagen está normalizada con una media cercana a 0 y una desviación estándar cercana a 1, y si la máscara está binarizada (contiene solo 0 y 1).
# Esta función no se utiliza directamente en el pipeline de entrenamiento.
#####################################

def verificar_normalizacion(img, msk):
    """
    Verifica si la imagen está normalizada con media cercana a 0 y std cercana a 1, y si la máscara está binarizada (contiene solo 0 y 1).
    """
    # Asegurar que img y msk no tengan dimensión extra del canal si es [D,H,W,1]
    if img.ndim == 4 and img.shape[-1] == 1:
        img = img[...,0]
    if msk.ndim == 4 and msk.shape[-1] == 1:
        msk = msk[...,0]

    media_imagen = np.mean(img)
    std_imagen = np.std(img)
    valores_unicos_mascara = np.unique(msk)
    print("\nVerificación de normalización:")
    print(f"Media de la imagen: {media_imagen:.4f}, Desviación estándar de la imagen: {std_imagen:.4f}")
    if abs(media_imagen) < 1e-3 and abs(std_imagen - 1.0) < 1e-3:
        print("La imagen parece estar correctamente normalizada (mean ~0, std ~1).")
    else:
        print("La imagen NO está correctamente normalizada.")
    print(f"Valores únicos en la máscara: {valores_unicos_mascara}")
    if np.array_equal(valores_unicos_mascara, [0, 1]):
        print("La máscara es binaria (0 y 1).")
    else:
        print("La máscara NO es binaria.")

###########################################
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!   ############################    OJO      ############################   !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Crea un conjunto de datos de TensorFlow que incluye tanto las imágenes originales como las aumentadas. Concatena el conjunto de datos original con un conjunto de datos
# aumentado y mezcla el conjunto combinado
# Esta función no se utiliza en el pipeline de entrenamiento proporcionado.
###########################################

def create_dataset_both(image_mask_pairs, batch_size=1):
    # Versión original (sin augment)
    ds_orig = create_dataset(image_mask_pairs, augment=False, batch_size=batch_size)
    # Versión aumentada (con augment)
    ds_aug  = create_dataset(image_mask_pairs, augment=True, batch_size=batch_size)
    # Concatenamos ambos
    ds_both = ds_orig.concatenate(ds_aug)
    # Re-barajamos para mezclar originales y aumentadas
    ds_both = ds_both.shuffle(buffer_size=len(image_mask_pairs)*2) # opcional
    return ds_both

#####################################
# Muestra una comparación entre una imagen original y su máscara versus una imagen aumentada y su máscara para un índice de corte dado.
# Esta función no se utiliza directamente en el pipeline de entrenamiento.
#####################################

def mostrar_imagenes_y_mascaras(original_imagen, original_mascara, augmented_imagen, augmented_mascara, slice_index=100):
    """
    Muestra comparaciones entre una imagen original y su máscara, vs. una imagen aumentada y su máscara.
    """
    # Asegurar que las imágenes no tengan dimensión extra del canal si es [D,H,W,1]
    def squeeze_channel(x):
        if x.ndim == 4 and x.shape[-1] == 1:
            return x[...,0]
        return x
    original_imagen = squeeze_channel(original_imagen)
    original_mascara = squeeze_channel(original_mascara)
    augmented_imagen = squeeze_channel(augmented_imagen)
    augmented_mascara = squeeze_channel(augmented_mascara)
    plt.figure(figsize=(16, 8))
    # Imagen original
    plt.subplot(2, 2, 1)
    #plt.imshow(original_imagen[:, :, slice_index], cmap='gray') #TRANSVERSAL
    plt.imshow(original_imagen[:, slice_index, :], cmap='gray') # CORONAL
    #plt.imshow(original_imagen[slice_index, :, :], cmap='gray') # SAGITAL
    plt.title('Imagen Original')
    plt.axis('off')
    # Máscara original
    plt.subplot(2, 2, 2)
    #plt.imshow(original_mascara[:, :, slice_index], cmap='gray') #TRANSVERSAL
    plt.imshow(original_mascara[:, slice_index, :], cmap='gray') # CORONAL
    #plt.imshow(original_mascara[slice_index, :, :], cmap='gray') # SAGITAL
    plt.title('Máscara Original')
    plt.axis('off')
    # Imagen aumentada
    plt.subplot(2, 2, 3)
    #plt.imshow(augmented_imagen[:, :, slice_index], cmap='gray') #TRANSVERSAL
    plt.imshow(augmented_imagen[:, slice_index, :], cmap='gray') # CORONAL
    #plt.imshow(augmented_imagen[slice_index, :, :], cmap='gray') # SAGITAL
    plt.title('Imagen Aumentada')
    plt.axis('off')
    # Máscara aumentada
    plt.subplot(2, 2, 4)
    #plt.imshow(augmented_mascara[:, :, slice_index], cmap='gray') #TRANSVERSAL
    plt.imshow(augmented_mascara[:, slice_index, :], cmap='gray') # CORONAL
    #plt.imshow(augmented_mascara[slice_index, :, :], cmap='gray') # SAGITAL
    plt.title('Máscara Aumentada')
    plt.axis('off')
    plt.show()

#####################################
# FUNCIONES DURANTE ENTRENAMIENTO
# Visualiza la imagen original, la máscara ground truth, la máscara predicha (media) y el mapa de incertidumbre (varianza) para un índice de corte dado.
# Esta función se utiliza durante la fase de evaluación para visualizar las predicciones y la incertidumbre del modelo.
#####################################

def mostrar_prediccion_error(imagen, mascara_real, mascara_predicha_media, varianza_predicha, slice_index=100):
    # Ajustar canales si es necesario
    def _squeeze_channel(x):
        if x.ndim == 4 and x.shape[-1] == 1:
            return x[..., 0]
        return x
    # Convertir a [D,H,W] si hace falta
    imagen = _squeeze_channel(imagen)
    mascara_real = _squeeze_channel(mascara_real)
    mascara_predicha_media = _squeeze_channel(mascara_predicha_media)
    varianza_predicha = _squeeze_channel(varianza_predicha)

    plt.figure(figsize=(16, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(imagen[:, :, slice_index], cmap='gray')
    plt.title('Imagen')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(mascara_real[:, :, slice_index], cmap='gray')
    plt.title('Máscara Real')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(mascara_predicha_media[:, :, slice_index], cmap='gray')
    plt.title('Máscara Predicha (Media)')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.imshow(varianza_predicha[:, :, slice_index], cmap='hot')
    plt.title('Incertidumbre (Desviación Estandar)')
    plt.axis('off')

    plt.show()

#**INSTANCIAMOS VALIDACIÓN CRUZADA**

In [ ]:
# Validación cruzada de 6 pliegues
kf = KFold(n_splits=6, shuffle=True, random_state=42)
# Inicializa KFold con 6 FOLDS, mezcla los datos antes de dividirlos y establece un estado aleatorio para la reproducibilidad.
# El modelo se entrenará y validará 6 veces, cada vez utilizando un fold diferente como conjunto de validación y los folds restantes como conjunto de entrenamiento.